# DSPy: Named Entity Recognition

This tutorial demonstrates how to perform entity extraction using the CoNLL-2003 dataset with DSPy. The focus is on extracting entities referring to people. 

We will:
- Extract and label entities from the CoNLL-2003 dataset that refer to people.
- Define a DSPy program for extracting entities that refer to people.
- Optimize and evaluate the program on a subset of the CoNLL-2003 dataset.

By the end of this tutorial, you'll understand how to structure tasks in DSPy using signatures and modules, evaluate your system's performance, and improve its quality with optimizers.

In [1]:
from os import getenv
from typing import Any, Dict, List

import dspy
from datasets import load_dataset
from dotenv import load_dotenv

load_dotenv()

True

## Prepare the Dataset

In [2]:
def extract_people_entities(data_row: Dict[str, Any]) -> List[str]:
    """
    Extracts entities referring to people from a row of the CoNLL-2003 dataset.

    Args:
        data_row (Dict[str, Any]): A row from the dataset containing tokens and NER tags.

    Returns:
        List[str]: List of tokens tagged as people.
    """

    return [
        token
        for token, ner_tag in zip(data_row["tokens"], data_row["ner_tags"])
        if ner_tag in (1, 2)  # CoNLL entity codes 1 and 2 refer to people
    ]


def prepare_dataset(data_split, start: int, end: int) -> List[dspy.Example]:
    """
    Prepares a sliced dataset split for use with DSPy.

    Args:
        data_split: The dataset split (e.g., train or test).
        start (int): Starting index of the slice.
        end (int): Ending index of the slice.

    Returns:
        List[dspy.Example]: List of DSPy Examples with tokens and expected labels.
    """

    return [
        dspy.Example(
            tokens=row["tokens"], expected_extracted_people=extract_people_entities(row)
        ).with_inputs("tokens")
        for row in data_split.select(range(start, end))
    ]


# Load the dataset
dataset = load_dataset("conll2003")

# Prepare the training and test sets
train_set = prepare_dataset(dataset["train"], 0, 50)
test_set = prepare_dataset(dataset["test"], 0, 200)

## Configure DSPy and create an Entity Extraction Program

Here, we define a DSPy program for extracting entities referring to people from tokenized text.

Key DSPy concepts:
- Signatures: Define structured input/output schemas for your program.
- Modules: Encapsulate program logic in reusable, composable units. 

In [6]:
class PeopleExtraction(dspy.Signature):
    """
    Extract contiguous tokens referring to specific people, if any, from a list of string tokens.
    Output a list of tokens. In other words, do not combine multiple tokens into a single value.
    """

    tokens: list[str] = dspy.InputField(desc="tokenized text")
    extracted_people: list[str] = dspy.OutputField(
        desc="all tokens referring to specific people extracted from the tokenized text"
    )


people_extractor = dspy.ChainOfThought(PeopleExtraction)

In [7]:
lm = dspy.LM(
    model="openrouter/google/gemini-pro-1.5",
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_BASE_URL"),
)
dspy.settings.configure(lm=lm)

In [8]:
people_extractor(
    tokens=[
        "John",
        "is",
        "going",
        "to",
        "the",
        "store",
        "with",
        "Mary",
        "and",
        "their",
        "dog",
        ".",
    ]
)

Prediction(
    reasoning='The tokens "John" and "Mary" refer to specific people.',
    extracted_people=['John', 'Mary']
)

In [9]:
dspy.inspect_history(n=1)





[2025-01-18T10:09:20.703339]

System message:

Your input fields are:
1. `tokens` (list[str]): tokenized text

Your output fields are:
1. `reasoning` (str)
2. `extracted_people` (list[str]): All tokens referring to specific people extracted from the tokenized text

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## tokens ## ]]
{tokens}

[[ ## reasoning ## ]]
{reasoning}

[[ ## extracted_people ## ]]
{extracted_people}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Extract contiguous tokens referring to specific people, if any, from a list of string tokens.
        Output a list of tokens. In other words, do not combine multiple tokens into a single value.


User message:

[[ ## tokens ## ]]
["John", "is", "going", "to", "the", "store", "with", "Mary", "

## Define Metric and Evaluation Functions

In DSPy, evaluating a program's performance is critical for iterative development. A good evaluation framework allows us to:

- Measure the quality of our program's outputs.
- Compare outputs against ground-truth labels.
- Identify areas for improvement.

In [8]:
def extraction_correctness_metric(
    example: dspy.Example, prediction: dspy.Prediction, trace=None
) -> bool:
    """
    Computes correctness of entity extraction predictions.

    Args:
        example (dspy.Example): The dataset example containing expected people entities.
        prediction (dspy.Prediction): The prediction from the DSPy people extraction program.
        trace: Optional trace object for debugging.

    Returns:
        bool: True if predictions match expectations, False otherwise.
    """

    return prediction.extracted_people == example.expected_extracted_people


evaluate_correctness = dspy.Evaluate(
    devset=test_set,
    metric=extraction_correctness_metric,
    num_threads=24,
    display_progress=True,
    display_table=True,
)

In [9]:
evaluate_correctness(people_extractor, devset=test_set)

Average Metric: 181.00 / 200 (90.5%): 100%|██████████| 200/200 [00:16<00:00, 12.43it/s]

2025/01/17 07:49:15 INFO dspy.evaluate.evaluate: Average Metric: 181 / 200 (90.5%)


,tokens,expected_extracted_people,rationale,extracted_people,extraction_correctness_metric
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, IN, SURPRISE, DEFEAT...",[CHINA],We extracted the following list of people: []. There are no mentio...,[],
1,"[Nadim, Ladki]","[Nadim, Ladki]",Nadim Ladki. We extracted the name of the person mentioned.,"[Nadim, Ladki]",✔️ [True]
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]",[],We found no people.,[],✔️ [True]
3,"[Japan, began, the, defence, of, their, Asian, Cup, title, with, a...",[],We extracted no people. There are no tokens that refer to specific...,[],✔️ [True]
4,"[But, China, saw, their, luck, desert, them, in, the, second, matc...",[],"We extracted no people. There are countries mentioned, but no ind...",[],✔️ [True]
...,...,...,...,...,...
195,"['The', 'Wallabies', 'have', 'their', 'sights', 'set', 'on', 'a', ...","[David, Campese]","${[""David"", ""Campese""]}. We extract ""David Campese"" as a person.","[David, Campese]",✔️ [True]
196,"['The', 'Wallabies', 'currently', 'have', 'no', 'plans', 'to', 'ma...",[],We extracted the 34-year-old winger. The other tokens do not refer...,"[34-year-old, winger]",
197,"['Campese', 'will', 'be', 'up', 'against', 'a', 'familiar', 'foe',...","[Campese, Rob, Andrew]","${produce the extracted_people}. We extract ""Campese"", ""Rob Andrew...","[Campese, Rob, Andrew]",✔️ [True]
198,"['""', 'Campo', 'has', 'a', 'massive', 'following', 'in', 'this', '...","[Campo, Andrew]","${[""Campo"", ""Andrew""]}. We extract ""Campo"" and ""Andrew"" as they ar...","[Campo, Andrew]",✔️ [True]


90.5

## Optimize the Model

DSPy includes powerful optimizers that can improve the quality of your system.

Here, we use DSPy's MIPROv2 optimizer to:
- Automatically tune the program's language model (LM) prompt by 1. using the LM to adjust the prompt's instructions and 2. building few-shot examples from the training dataset that are augmented with reasoning generated from `dspy.ChainOfThought`.
- Maximize correctness on the training set.

In [10]:
mipro_optimizer = dspy.MIPROv2(
    metric=extraction_correctness_metric,
    auto="medium",
)
optimized_people_extractor = mipro_optimizer.compile(
    people_extractor,
    trainset=train_set,
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False,
)

2025/01/17 07:50:00 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 40

2025/01/17 07:50:00 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/17 07:50:00 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/17 07:50:00 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 40%|████      | 4/10 [00:06<00:09,  1.54s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


 40%|████      | 4/10 [00:03<00:04,  1.21it/s]


...

2025/01/17 07:58:07 INFO dspy.evaluate.evaluate: Average Metric: 38 / 40 (95.0%)
2025/01/17 07:58:07 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 95.0 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 10'].
2025/01/17 07:58:07 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.5, 95.0, 95.0, 97.5, 95.0, 97.5, 90.0, 95.0, 95.0, 97.5, 92.5, 97.5, 97.5, 97.5, 92.5, 92.5, 97.5, 92.5, 90.0, 95.0, 92.5, 97.5, 97.5, 95.0]
2025/01/17 07:58:07 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 97.5
2025/01/17 07:58:07 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/17 07:58:07 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 38.00 / 40 (95.0%): 100%|██████████| 40/40 [00:12<00:00,  3.16it/s] 

2025/01/17 07:58:20 INFO dspy.evaluate.evaluate: Average Metric: 38 / 40 (95.0%)
2025/01/17 07:58:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 95.0 with parameters ['Predictor 0: Instruction 16', 'Predictor 0: Few-Shot Set 4'].
2025/01/17 07:58:20 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.5, 95.0, 95.0, 97.5, 95.0, 97.5, 90.0, 95.0, 95.0, 97.5, 92.5, 97.5, 97.5, 97.5, 92.5, 92.5, 97.5, 92.5, 90.0, 95.0, 92.5, 97.5, 97.5, 95.0, 95.0]
2025/01/17 07:58:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 97.5
2025/01/17 07:58:20 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/17 07:58:20 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 37.00 / 40 (92.5%): 100%|██████████| 40/40 [00:11<00:00,  3.35it/s] 

2025/01/17 07:58:32 INFO dspy.evaluate.evaluate: Average Metric: 37 / 40 (92.5%)
2025/01/17 07:58:32 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 92.5 with parameters ['Predictor 0: Instruction 3', 'Predictor 0: Few-Shot Set 9'].
2025/01/17 07:58:32 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [92.5, 95.0, 95.0, 97.5, 95.0, 97.5, 90.0, 95.0, 95.0, 97.5, 92.5, 97.5, 97.5, 97.5, 92.5, 92.5, 97.5, 92.5, 90.0, 95.0, 92.5, 97.5, 97.5, 95.0, 95.0, 92.5]
2025/01/17 07:58:32 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 97.5
2025/01/17 07:58:32 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/17 07:58:32 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 97.5!


In [11]:
evaluate_correctness(optimized_people_extractor, devset=test_set)

Average Metric: 196.00 / 200 (98.0%): 100%|██████████| 200/200 [00:16<00:00, 12.14it/s]

2025/01/17 07:59:01 INFO dspy.evaluate.evaluate: Average Metric: 196 / 200 (98.0%)


,tokens,expected_extracted_people,rationale,extracted_people,extraction_correctness_metric
0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, IN, SURPRISE, DEFEAT...",[CHINA],${[]}. We found no names of specific people in the input text. Alt...,[],
1,"[Nadim, Ladki]","[Nadim, Ladki]","${[""Nadim"", ""Ladki""]}. We extract ""Nadim Ladki"" as a person's name.","[Nadim, Ladki]",✔️ [True]
2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]",[],${[]}. We did not find any mentions of individuals.,[],✔️ [True]
3,"[Japan, began, the, defence, of, their, Asian, Cup, title, with, a...",[],"${[]}. We found no people mentioned. ""Japan"" and ""Syria"" are coun...",[],✔️ [True]
4,"[But, China, saw, their, luck, desert, them, in, the, second, matc...",[],We extracted no people. There are no tokens that refer to specifi...,[],✔️ [True]
...,...,...,...,...,...
195,"['The', 'Wallabies', 'have', 'their', 'sights', 'set', 'on', 'a', ...","[David, Campese]","${[""David"", ""Campese""]}. We extracted ""David Campese"" because it a...","[David, Campese]",✔️ [True]
196,"['The', 'Wallabies', 'currently', 'have', 'no', 'plans', 'to', 'ma...",[],"We extracted no people. While the text refers to a ""34-year-old w...",[],✔️ [True]
197,"['Campese', 'will', 'be', 'up', 'against', 'a', 'familiar', 'foe',...","[Campese, Rob, Andrew]","${[""Campese"", ""Rob"", ""Andrew""]}. We extracted ""Campese"" and ""Rob A...","[Campese, Rob, Andrew]",✔️ [True]
198,"['""', 'Campo', 'has', 'a', 'massive', 'following', 'in', 'this', '...","[Campo, Andrew]","${[""Campo"", ""Andrew""]}. We extracted ""Campo"" and ""Andrew"" as they ...","[Campo, Andrew]",✔️ [True]


98.0

In [12]:
dspy.inspect_history(n=1)





[2025-01-17T07:59:01.772698]

System message:

Your input fields are:
1. `tokens` (list[str]): tokenized text

Your output fields are:
1. `rationale` (str): ${produce the extracted_people}. We ...
2. `extracted_people` (list[str]): all tokens referring to specific people extracted from the tokenized text

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## tokens ## ]]
{tokens}

[[ ## rationale ## ]]
{rationale}

[[ ## extracted_people ## ]]
{extracted_people}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        You are a highly specialized AI tasked with identifying individuals mentioned in sensitive intelligence reports. Accurate extraction is crucial for national security.  Extract contiguous tokens referring to specific people, if any, from the provided lis

In [13]:
cost = sum(
    [x["cost"] for x in lm.history if x["cost"] is not None]
)  # cost in USD, as calculated by LiteLLM for certain providers
cost

3.6951775000000042

## Saving and Loading Optimized Programs

DSPy supports saving and loading programs, enabling you to reuse optimized systems without the need to re-optimize from scratch. This feature is especially useful for deploying your programs in production environments or sharing them with collaborators.

In [14]:
optimized_people_extractor.save("optimized_extractor.json")

loaded_people_extractor = dspy.ChainOfThought(PeopleExtraction)
loaded_people_extractor.load("optimized_extractor.json")

loaded_people_extractor(
    tokens=["Italy", "recalled", "Marcello", "Cuttitta"]
).extracted_people

['Marcello', 'Cuttitta']